<a href="https://colab.research.google.com/github/polarbeast8/Stockprediction/blob/main/stockprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import pandas_datareader as pdr
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly import subplots
import plotly.figure_factory as ff
from ipywidgets import interact
import ipywidgets as widgets
import math
from sklearn.metrics import mean_squared_error
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

In [37]:
tickers = ['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOGL']
stock = pdr.get_data_yahoo(tickers, start='2011-01-01', end='2021-01-01')
stock

[*********************100%***********************]  5 of 5 completed


Adj Close                           ...      Volume                   
                  AAPL         AMZN          FB  ...          FB    GOOGL      NFLX
Date                                             ...                               
2011-01-03   10.091492   184.220001         NaN  ...         NaN  4725670  39956000
2011-01-04   10.144159   185.009995         NaN  ...         NaN  3645351  44065700
2011-01-05   10.227141   187.419998         NaN  ...         NaN  5059535  31799600
2011-01-06   10.218874   185.860001         NaN  ...         NaN  4111484  31991400
2011-01-07   10.292057   185.490005         NaN  ...         NaN  4198198  22155000
...                ...          ...         ...  ...         ...      ...       ...
2020-12-24  131.161407  3172.689941  267.399994  ...   6702000.0   465600   1144000
2020-12-28  135.852509  3283.959961  277.000000  ...  23299700.0  1382500   2891900
2020-12-29  134.043655  3322.000000  276.779999  ...  16383000.0   986300   4022400
2020-12-30  132.900696  3285.850098  271.869995  ...  11803800.0  1051300   1876300
2020-12-31  131.877014  3256.929932  273.160004  ...  12900400.0  1053500   5392300

[2517 rows x 30 columns]

In [38]:
stock.columns

MultiIndex([('Adj Close',  'AAPL'),
            ('Adj Close',  'AMZN'),
            ('Adj Close',    'FB'),
            ('Adj Close', 'GOOGL'),
            ('Adj Close',  'NFLX'),
            (    'Close',  'AAPL'),
            (    'Close',  'AMZN'),
            (    'Close',    'FB'),
            (    'Close', 'GOOGL'),
            (    'Close',  'NFLX'),
            (     'High',  'AAPL'),
            (     'High',  'AMZN'),
            (     'High',    'FB'),
            (     'High', 'GOOGL'),
            (     'High',  'NFLX'),
            (      'Low',  'AAPL'),
            (      'Low',  'AMZN'),
            (      'Low',    'FB'),
            (      'Low', 'GOOGL'),
            (      'Low',  'NFLX'),
            (     'Open',  'AAPL'),
            (     'Open',  'AMZN'),
            (     'Open',    'FB'),
            (     'Open', 'GOOGL'),
            (     'Open',  'NFLX'),
            (   'Volume',  'AAPL'),
            (   'Volume',  'AMZN'),
            (   'Volume',   

In [39]:
stock.columns = [c[0]+"_"+c[1] for c in stock.columns]
stock

,Adj Close_AAPL,Adj Close_AMZN,Adj Close_FB,Adj Close_GOOGL,Adj Close_NFLX,Close_AAPL,Close_AMZN,Close_FB,Close_GOOGL,Close_NFLX,High_AAPL,High_AMZN,High_FB,High_GOOGL,High_NFLX,Low_AAPL,Low_AMZN,Low_FB,Low_GOOGL,Low_NFLX,Open_AAPL,Open_AMZN,Open_FB,Open_GOOGL,Open_NFLX,Volume_AAPL,Volume_AMZN,Volume_FB,Volume_GOOGL,Volume_NFLX
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-01-03,10.091492,184.220001,NaN,302.477478,25.487143,11.770357,184.220001,NaN,302.477478,25.487143,11.795000,186.000000,NaN,303.098083,25.838572,11.601429,181.210007,NaN,298.538544,24.785713,11.630000,181.369995,NaN,298.538544,25.000000,445138400,5331400,NaN,4725670,39956000
2011-01-04,10.144159,185.009995,NaN,301.361359,25.910000,11.831786,185.009995,NaN,301.361359,25.910000,11.875000,187.699997,NaN,303.393402,26.414286,11.719643,183.779999,NaN,300.360352,25.469999,11.872857,186.149994,NaN,303.113098,25.907143,309080800,5031800,NaN,3645351,44065700
2011-01-05,10.227141,187.419998,NaN,304.839844,25.675714,11.928571,187.419998,NaN,304.839844,25.675714,11.940714,187.449997,NaN,305.470459,26.112858,11.767857,184.070007,NaN,300.325317,25.530001,11.769643,184.100006,NaN,300.335327,25.878571,255519600,3418800,NaN,5059535,31799600
2011-01-06,10.218874,185.860001,NaN,307.057068,25.427143,11.918929,185.860001,NaN,307.057068,25.427143,11.973214,187.410004,NaN,309.524536,25.670000,11.889286,185.250000,NaN,305.330322,25.092857,11.954286,186.500000,NaN,305.645660,25.168571,300428800,3179700,NaN,4111484,31991400
2011-01-07,10.292057,185.490005,NaN,308.528534,25.614286,12.004286,185.490005,NaN,308.528534,25.614286,12.012500,188.449997,NaN,309.434448,25.792856,11.853571,183.740005,NaN,305.370361,25.197144,11.928214,187.880005,NaN,308.263275,25.524286,311931200,5221700,NaN,4198198,22155000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,131.161407,3172.689941,267.399994,1734.160034,513.969971,131.970001,3172.689941,267.399994,1734.160034,513.969971,133.460007,3202.000000,270.399994,1742.410034,519.349976,131.100006,3169.000000,266.200012,1724.349976,512.210022,131.320007,3193.899902,268.880005,1729.000000,515.119995,54930100,1451900,6702000.0,465600,1144000
2020-12-28,135.852509,3283.959961,277.000000,1773.959961,519.119995,136.690002,3283.959961,277.000000,1773.959961,519.119995,137.339996,3304.000000,277.299988,1787.000000,523.659973,133.509995,3172.689941,265.660004,1741.819946,507.130005,133.990005,3194.000000,268.739990,1744.910034,516.429993,124486200,5686800,23299700.0,1382500,2891900
2020-12-29,134.043655,3322.000000,276.779999,1757.760010,530.869995,134.869995,3322.000000,276.779999,1757.760010,530.869995,138.789993,3350.649902,280.510010,1788.469971,536.549988,134.339996,3281.219971,276.279999,1755.109985,515.479980,138.050003,3309.939941,276.950012,1787.229980,519.900024,121047300,4872900,16383000.0,986300,4022400


In [40]:
stock = stock.reset_index().rename({'index':'Date1'},axis='columns')
stock

,Date,Adj Close_AAPL,Adj Close_AMZN,Adj Close_FB,Adj Close_GOOGL,Adj Close_NFLX,Close_AAPL,Close_AMZN,Close_FB,Close_GOOGL,Close_NFLX,High_AAPL,High_AMZN,High_FB,High_GOOGL,High_NFLX,Low_AAPL,Low_AMZN,Low_FB,Low_GOOGL,Low_NFLX,Open_AAPL,Open_AMZN,Open_FB,Open_GOOGL,Open_NFLX,Volume_AAPL,Volume_AMZN,Volume_FB,Volume_GOOGL,Volume_NFLX
0,2011-01-03,10.091492,184.220001,NaN,302.477478,25.487143,11.770357,184.220001,NaN,302.477478,25.487143,11.795000,186.000000,NaN,303.098083,25.838572,11.601429,181.210007,NaN,298.538544,24.785713,11.630000,181.369995,NaN,298.538544,25.000000,445138400,5331400,NaN,4725670,39956000
1,2011-01-04,10.144159,185.009995,NaN,301.361359,25.910000,11.831786,185.009995,NaN,301.361359,25.910000,11.875000,187.699997,NaN,303.393402,26.414286,11.719643,183.779999,NaN,300.360352,25.469999,11.872857,186.149994,NaN,303.113098,25.907143,309080800,5031800,NaN,3645351,44065700
2,2011-01-05,10.227141,187.419998,NaN,304.839844,25.675714,11.928571,187.419998,NaN,304.839844,25.675714,11.940714,187.449997,NaN,305.470459,26.112858,11.767857,184.070007,NaN,300.325317,25.530001,11.769643,184.100006,NaN,300.335327,25.878571,255519600,3418800,NaN,5059535,31799600
3,2011-01-06,10.218874,185.860001,NaN,307.057068,25.427143,11.918929,185.860001,NaN,307.057068,25.427143,11.973214,187.410004,NaN,309.524536,25.670000,11.889286,185.250000,NaN,305.330322,25.092857,11.954286,186.500000,NaN,305.645660,25.168571,300428800,3179700,NaN,4111484,31991400
4,2011-01-07,10.292057,185.490005,NaN,308.528534,25.614286,12.004286,185.490005,NaN,308.528534,25.614286,12.012500,188.449997,NaN,309.434448,25.792856,11.853571,183.740005,NaN,305.370361,25.197144,11.928214,187.880005,NaN,308.263275,25.524286,311931200,5221700,NaN,4198198,22155000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2512,2020-12-24,131.161407,3172.689941,267.399994,1734.160034,513.969971,131.970001,3172.689941,267.399994,1734.160034,513.969971,133.460007,3202.000000,270.399994,1742.410034,519.349976,131.100006,3169.000000,266.200012,1724.349976,512.210022,131.320007,3193.899902,268.880005,1729.000000,515.119995,54930100,1451900,6702000.0,465600,1144000
2513,2020-12-28,135.852509,3283.959961,277.000000,1773.959961,519.119995,136.690002,3283.959961,277.000000,1773.959961,519.119995,137.339996,3304.000000,277.299988,1787.000000,523.659973,133.509995,3172.689941,265.660004,1741.819946,507.130005,133.990005,3194.000000,268.739990,1744.910034,516.429993,124486200,5686800,23299700.0,1382500,2891900
2514,2020-12-29,134.043655,3322.000000,276.779999,1757.760010,530.869995,134.869995,3322.000000,276.779999,1757.760010,530.869995,138.789993,3350.649902,280.510010,1788.469971,536.549988,134.339996,3281.219971,276.279999,1755.109985,515.479980,138.050003,3309.939941,276.950012,1787.229980,519.900024,121047300,4872900,16383000.0,986300,4022400
2515,2020-12-30,132.900696,3285.850098,271.869995,1736.250000,524.590027,133.720001,3285.850098,271.869995,1736.250000,524.590027,135.990005,3342.100098,278.079987,1767.760010,533.260010,133.399994,3282.469971,271.709991,1728.000000,523.690002,135.580002,3341.000000,277.950012,1765.000000,530.130005,96452100,3209300,11803800.0,1051300,1876300


In [41]:
df = stock[['Date','Close_FB','Close_AMZN','Close_AAPL','Close_NFLX','Close_GOOGL']]

In [42]:
df.rename(columns={'Close_FB':'FB','Close_AMZN':'AMZN','Close_AAPL':'AAPL','Close_NFLX':'NFLX','Close_GOOGL':'GOOGL'}, inplace=True)
df

,Date,FB,AMZN,AAPL,NFLX,GOOGL
0,2011-01-03,NaN,184.220001,11.770357,25.487143,302.477478
1,2011-01-04,NaN,185.009995,11.831786,25.910000,301.361359
2,2011-01-05,NaN,187.419998,11.928571,25.675714,304.839844
3,2011-01-06,NaN,185.860001,11.918929,25.427143,307.057068
4,2011-01-07,NaN,185.490005,12.004286,25.614286,308.528534
...,...,...,...,...,...,...
2512,2020-12-24,267.399994,3172.689941,131.970001,513.969971,1734.160034
2513,2020-12-28,277.000000,3283.959961,136.690002,519.119995,1773.959961
2514,2020-12-29,276.779999,3322.000000,134.869995,530.869995,1757.760010
2515,2020-12-30,271.869995,3285.850098,133.720001,524.590027,1736.250000


In [43]:
df.isnull().sum()

Date       0
FB       347
AMZN       0
AAPL       0
NFLX       0
GOOGL      0
dtype: int64

In [44]:
def stock_plot(stock_name):
  fig = px.line(df,x='Date',y=stock_name)
  fig.show()

In [45]:
interact(stock_plot,stock_name=['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOGL'],width=600,height=150)

interactive(children=(Dropdown(description='stock_name', options=('FB', 'AMZN', 'AAPL', 'NFLX', 'GOOGL'), valu…

<function __main__.stock_plot>

# LSTM

In [46]:
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return np.array(dataX), np.array(dataY)

In [47]:
# pred_df
# pred_df3
def lstm_pred(stock_name):
  global pred_df, pred_df3
  scaler = MinMaxScaler(feature_range=(0,1))
  scaled_df= scaler.fit_transform(df[[stock_name]].values.reshape(-1,1))
  training_size=int(len(scaled_df)*0.75)
  test_size=len(scaled_df)-training_size
  train_data,test_data=scaled_df[0:training_size,:],scaled_df[training_size:len(scaled_df),:1]
  training_size,test_size
  time_step = 100
  X_train, y_train = create_dataset(train_data, time_step)
  X_test, ytest = create_dataset(test_data, time_step)
  X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
  X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
  model=Sequential()
  model.add(LSTM(50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
  model.add(LSTM(50,return_sequences=True))
  model.add(LSTM(50))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error',optimizer='adam')
  model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)
  train_predict=model.predict(X_train)
  test_predict=model.predict(X_test)
  train_predict=scaler.inverse_transform(train_predict)
  test_predict=scaler.inverse_transform(test_predict)
  look_back=100
  trainPredictPlot = np.empty_like(scaled_df)
  trainPredictPlot[:, :] = np.nan
  trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
  testPredictPlot = np.empty_like(scaled_df)
  testPredictPlot[:, :] = np.nan
  testPredictPlot[len(train_predict)+(look_back*2)+1:len(scaled_df)-1, :] = test_predict
  new_df = pd.DataFrame(data=scaler.inverse_transform(scaled_df), columns=["Real"])
  new_df["Train"] = trainPredictPlot
  new_df["Test"] = testPredictPlot
  new_df["Date"] = stock["Date"]
  fig=px.line(new_df,x="Date", y="Real")
  fig.add_scatter(x=new_df["Date"],y=new_df["Train"], mode='lines', name='Train')
  fig.add_scatter(x=new_df["Date"],y=new_df["Test"], mode='lines', name="Test")
  fig['data'][0]['showlegend']=True
  fig['data'][0]['name']='Real Value'
  fig.show()
  #Next 3 days prediction
  x_input=test_data[len(test_data)-100:].reshape(1,-1)
  x_input.shape
  temp_input=list(x_input)
  temp_input=temp_input[0].tolist()
  lst_output=[]
  n_steps=100
  i=0
  while(i<3):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
  day_new=np.arange(1,101)
  day_pred=np.arange(101,104)
  pred_df = pd.DataFrame(data=scaler.inverse_transform(scaled_df), columns=["Real"])
  pred_df["Date"] = stock["Date"]
  pred_df3 = pd.DataFrame(data=scaler.inverse_transform(lst_output), columns=["Predicted"])
  pred_df3["Date"] = pd.date_range(start='1/1/2021',periods=len(pred_df3), freq='D')
  fig1=px.line(pred_df,x="Date", y="Real")
  fig1.add_scatter(x=pred_df3["Date"],y=pred_df3["Predicted"], mode='lines', name='Predicted')
  fig1['data'][0]['showlegend']=True
  fig1['data'][0]['name']='Real Value'
  fig1.show()



In [48]:
lstm_pred("AAPL")

Epoch 1/100
28/28 [==============================] - 12s 252ms/step - loss: 0.0024 - val_loss: 0.0046
Epoch 2/100
28/28 [==============================] - 6s 200ms/step - loss: 1.6905e-04 - val_loss: 0.0033
Epoch 3/100
28/28 [==============================] - 5s 195ms/step - loss: 1.0003e-04 - val_loss: 0.0034
Epoch 4/100
28/28 [==============================] - 5s 195ms/step - loss: 9.0284e-05 - val_loss: 0.0036
Epoch 5/100
28/28 [==============================] - 6s 198ms/step - loss: 8.7293e-05 - val_loss: 0.0030
Epoch 6/100
28/28 [==============================] - 5s 195ms/step - loss: 8.5320e-05 - val_loss: 0.0023
Epoch 7/100
28/28 [==============================] - 5s 197ms/step - loss: 8.6843e-05 - val_loss: 0.0029
Epoch 8/100
28/28 [==============================] - 5s 196ms/step - loss: 8.1968e-05 - val_loss: 0.0026
Epoch 9/100
28/28 [==============================] - 6s 200ms/step - loss: 8.1344e-05 - val_loss: 0.0026
Epoch 10/100
28/28 [==============================] - 5s 1

[0.90661263]
101
1 day input [0.81120729 0.82715262 0.82633541 0.82394359 0.8315575  0.83271351
 0.85318337 0.90177677 0.91363607 0.90540429 0.9189379  0.90687927
 0.90526481 0.93900904 0.97998853 0.95782453 0.87395212 0.87458995
 0.80969246 0.84556946 0.81503414 0.80315488 0.82994304 0.83137812
 0.80419131 0.78992023 0.7620159  0.78784738 0.80164006 0.7642483
 0.77301822 0.80538722 0.82675396 0.81981772 0.83353072 0.84134395
 0.81128697 0.83903187 0.8124032  0.8277107  0.82683371 0.84277904
 0.90201594 0.87570612 0.87642369 0.87259679 0.85912296 0.83488611
 0.84708428 0.84198178 0.83305238 0.82739179 0.82747153 0.83982913
 0.79677673 0.82962413 0.77812072 0.77740315 0.79071754 0.82667422
 0.85920271 0.85649203 0.83759679 0.83480637 0.86287013 0.86063779
 0.86103645 0.86932803 0.86207287 0.85123004 0.85609337 0.84572888
 0.8179043  0.82842821 0.83528471 0.83974938 0.85936219 0.88862186
 0.89149202 0.89037586 0.8848747  0.8968337  0.90185645 0.88112754
 0.89276762 0.88615035 0.88112754 

In [49]:
# interact(lstm_pred,stock_name=['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOGL'])

# Prophet

In [50]:
prophet_df = df
orig = df
def prophet_pred(st_name):
  global prophet_df
  prophet_df = df[["Date"]]
  prophet_df["Date"] = pd.to_datetime(prophet_df["Date"], dayfirst=True)
  prophet_df["st_price"] = df[st_name]
  prophet_df.columns = ['ds','y']

  m = Prophet(interval_width = 0.98)
  m.fit(prophet_df)
  future = m.make_future_dataframe(periods = 30)
  forecast = m.predict(future)
  fig = px.line(x=forecast['ds'], y=forecast[('yhat')], labels= {'x' : 'Date', 'y' : 'Price'})
  fig.add_scatter(x=prophet_df["ds"],y=prophet_df["y"], mode='lines', name='Original Value')
  fig['data'][0]['showlegend']=True
  fig['data'][0]['name']='Predicted Value'
  # df_cv = cross_validation(m, initial='800 days', period='2 days', horizon = '30 days')
  # df_p = performance_metrics(df_cv)
  # print(df_p)
  global orig
  orig = forecast
  # print(forecast)
  fig.show()
  
  # return forecast



In [51]:
interact(prophet_pred,st_name=['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOGL'])


interactive(children=(Dropdown(description='st_name', options=('FB', 'AMZN', 'AAPL', 'NFLX', 'GOOGL'), value='…

<function __main__.prophet_pred>

In [52]:
pred_df3["Date"] = pd.date_range(start='1/1/2021',periods=len(pred_df3), freq='D')
fig1=px.line(pred_df,x="Date", y="Real")
fig1.add_scatter(x=pred_df3["Date"],y=pred_df3["Predicted"], mode='lines', name='Predicted')
fig1['data'][0]['showlegend']=True
fig1['data'][0]['name']='Real Value'
fig1.show()

In [53]:
pred_df


,Real,Date
0,11.770357,2011-01-03
1,11.831786,2011-01-04
2,11.928571,2011-01-05
3,11.918929,2011-01-06
4,12.004286,2011-01-07
...,...,...
2512,131.970001,2020-12-24
2513,136.690002,2020-12-28
2514,134.869995,2020-12-29
2515,133.720001,2020-12-30


In [55]:
fig = px.line(x=orig['ds'], y=orig[('yhat')], labels= {'x' : 'Date', 'y' : 'Price'})
fig.add_scatter(x=prophet_df["ds"],y=prophet_df["y"], mode='lines', name='Original Value')
pred_df3["Date"] = pd.date_range(start='1/1/2021',periods=len(pred_df3), freq='D')
# fig.add_scatter(x=pred_df["Date"],y=pred_df["Real"], mode='lines', name='LSTM Train Test')
# fig.add_scatter(x=pred_df3["Date"],y=pred_df3["Predicted"], mode='lines', name='LSTM Predicted')
fig['data'][0]['showlegend']=True
fig['data'][0]['name']='Predicted Value'
fig.show()
